In [1]:
%run scripts/helper.py

In [2]:
magazino_train = load_data('./train.csv/train.csv', index_col='id')
magazino_test = load_data('./test.csv', index_col='id')

In [3]:
magazino_train.columns = trim_column(magazino_train)
magazino_test.columns = trim_column(magazino_test)

In [4]:
features = return_features(magazino_train)

In [5]:
from sklearn.preprocessing import StandardScaler

In [6]:
scl = StandardScaler()
features_scaled = scl.fit_transform(features, magazino_train.shares)

In [7]:
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE

In [8]:
Xt, Xv, yt, yv = split_data(features_scaled, magazino_train.shares)

In [9]:
lr = LinearRegression()
selector = RFE(lr)
selector = selector.fit(Xt, yt)

In [10]:
model_estimated = selector.estimator_

In [11]:
Xt_selected = Xt[:, selector.support_]

In [12]:
gbr_model = build_model(Xt_selected, yt)

In [59]:
yt_pred = model_estimated.predict(Xt_selected)

In [60]:
print 'RMSE score on training score %0.4f ' %(RMSE(yt, yt_pred))

RMSE score on training score 12032.4957 


In [61]:
Xv_selected = Xv[:, selector.support_]

In [63]:
yv_pred = model_estimated.predict(Xv_selected)

In [64]:
print 'RMSE score on test score %0.4f ' %(RMSE(yv, yv_pred))

RMSE score on test score 7861.3728 


In [25]:
# model = lr.fit(features_scaled, magazino_train.shares)

In [65]:
features_test = return_features(magazino_test)

In [66]:
features_test_scaled = scl.transform(features_test)

In [67]:
features_test_scaled_selected = features_test_scaled[:, selector.support_]

In [70]:
final_preds = model_estimated.predict(features_test_scaled_selected)

In [71]:
make_submission(magazino_test.index.values, final_preds, 'selected_features_linear_regression.csv')